## Audio Transcription
This file transcribes the speech in an audio file with the following steps.
1. Conversion of MP4 to WAV
2. Transcription of audio file

## Step 1: Conversion of MP4 to WAV
This is the first in the transcription. The function below does the following
1. Receive a mp4 file 
2. Convert file to WAV file 
3. Store WAV file in an assigned file path

In [3]:
from moviepy.editor import AudioFileClip

def convert_audio_to_wav(input_file, output_file):
    audio_clip = AudioFileClip(input_file)
    audio_clip.write_audiofile(output_file, codec='pcm_s16le', fps=16000)

if __name__ == "__main__":
    input_audio_file = "10th_Meeting_3rd_Session_Open-ended_Working_Group_on_Reducing_Space_Threats-Kaltura-1_9nuc6rbb-hls-audio-English.mp4"  
    output_wav_file = "meeting10.wav"  

    convert_audio_to_wav(input_audio_file, output_wav_file)

chunk:   0%|          | 21/86714 [01:32<105:55:01,  4.40s/it, now=None]

MoviePy - Writing audio in /Users/pelumioluwaabiola/Downloads/meeting10.wav


chunk:   0%|          | 21/86714 [03:04<212:02:05,  8.80s/it, now=None]

MoviePy - Done.


## Step 2: Transcription of Audio File

This includes the step below 
1. import microsoft speech SDK and input credential details (api_key and location region)
2. Open text file 
3. Define stop fuction to continue speech recognition till the audio is done
4. Define transcription function to read transcription results, get the audio duration of speech and write both duration and transcription into text file immediately
5. Once speech recognition is done, save file and close transcription text

In [5]:
import azure.cognitiveservices.speech as speechsdk
import json
import time
from dotenv import dotenv_values

audio_filename = "/Users/pelumioluwaabiola/Downloads/meeting10.wav"
f = open('/Users/pelumioluwaabiola/Downloads/transcription10.txt', 'w')  # Open a text file at the beginning

credentials = dotenv_values('credential.env')
speech_key = credentials['speech_key']
service_region = credentials['service_region']


speech_config = speechsdk.SpeechConfig(subscription=speech_key, region=service_region)
speech_config.output_format = speechsdk.OutputFormat.Detailed


audio_input = speechsdk.AudioConfig(filename=audio_filename)
speech_recognizer = speechsdk.SpeechRecognizer(speech_config=speech_config, audio_config=audio_input)

done = False

def stop_cb(evt):
    global done
    print('CLOSING on {}'.format(evt))
    done = True
    if evt.reason == speechsdk.ResultReason.Canceled:
        cancellation_details = speechsdk.CancellationDetails.from_result(evt.result)
        print("Error details: {}".format(cancellation_details.error_details))
        

    speech_recognizer.stop_continuous_recognition()
    done = True
    f.close() # Close the file when recognition is done
    print("Recognition stopped and file closed.")  


def write_transcription(evt):
    result_json = json.loads(evt.result.json)
    offset = result_json['Offset']
    #convert offset to seconds
    converted_offset =  offset / 10000000
    # convert offset to hours, minutes, and seconds
    hours, remainder = divmod(converted_offset, 3600)
    minutes, seconds = divmod(remainder, 60)

    # format the start_time
    start_time = f"{int(hours):02d}:{int(minutes):02d}:{int(seconds):02d}"

    f.write(f"{start_time}: {result_json['DisplayText']}\n")  # Write to the file
    f.flush()  # Ensure the content is written to the file immediately
    print(f"Transcribed: {start_time}: {result_json['DisplayText']}")  


speech_recognizer.recognized.connect(write_transcription)
speech_recognizer.session_stopped.connect(stop_cb)
speech_recognizer.canceled.connect(stop_cb)

speech_recognizer.start_continuous_recognition()
while not done:
    time.sleep(.5)

Transcribed: 00:00:04: Good afternoon, Your Excellencies.
Transcribed: 00:00:08: Distinguished Delegates.
Transcribed: 00:00:11: We will now reconvene and continue our debates on topic 10.
Transcribed: 00:00:16: And we still have some speakers pending from this morning's debate. And the next speaker will be the distinguished representative of the United States. You have the floor, Sir.
Transcribed: 00:00:35: Thank you, Mr. Chairman. Thank you for the opportunity to speak first after lunch and I appreciate my Russian colleagues agreeing with everything I am about to say.
Transcribed: 00:00:50: Mr. Chairman, sorry, if you prefer, you can delay your your intervention. No, no, no, I'm ready now. Go ahead.
Transcribed: 00:01:01: Marks, Mr. Chairman, as some delegations here have noted the task before the open-ended working group.
Transcribed: 00:01:10: Is to make recommendations on possible norms, rules, and principles of responsible behavior relating to threats by states to space systems, 